# Experimental details - prompt ingredients

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

import pandas as pd

import prompt_library as pl
import utils as ut

## Example prompt

In [3]:
example_prompts_pd = pd.read_json("../results/data/exp_1_Nurse_choices.json")
example_prompts = example_prompts_pd[example_prompts_pd['permutation'] == 0][["condition", "prompt"]].drop_duplicates().to_dict(orient="records")
ii = 1
assert example_prompts[ii]['condition'] == 'DECOY'

example_prompt = example_prompts[ii]['prompt']
print(example_prompt)

ut.save_verbatim_text(text=example_prompt, tag="example_prompt")


Act as an experienced and fair recruiter with excellent analytical skills. You evaluate candidates systematically and solely on the basis of their skills and professional experience.

**TASK**: You have to hire a *Nurse*. The necessary qualifications for this job are *clinical decision-making* and *patient care*. These qualifications are *EQUALLY* important. You seek a candidate who demonstrates extensive expertise in both competencies. A valid working permit is required for this job.

You need to choose exactly *one* from the following candidates:
###
- A: The candidate's *clinical decision-making* experience is 3 years and their *patient care* experience is 6 years. The candidate holds a valid working permit.
- B: The candidate's *clinical decision-making* experience is 6 years and their *patient care* experience is 3 years. The candidate holds a valid working permit.
- C: The candidate's *clinical decision-making* experience is 2 years and their *patient care* experience is 5 years.

## Job openings

In [4]:
job_opening_pd = pd.DataFrame(pl.jobs_openings.values())

renaming_dict = {
    "job_title": "Job title"
}

for i in [1,2]:
    job_opening_pd[f"Required qualification {i}"] = job_opening_pd[f"dim_{i}"].apply(lambda row: f"{row['label']} {row['type']} [{row['unit']}]")

job_opening_pd[f"Occupation classification"] = job_opening_pd["tags"].apply(lambda row: ", ".join([v for v in row if v not in [pl.num_num, pl.cat_num]]))
job_opening_pd = job_opening_pd.rename(columns=renaming_dict)[
    ["Job title", "Required qualification 1", "Required qualification 2", "Occupation classification"]
    ]

display(job_opening_pd) 

ut.save_latex_table(
    df=job_opening_pd, 
    tag="job_openings", 
    caption=f"Job openings specified by a job title and two required and equally important qualifications. The jobs are classified by stereotypical gender dominance and collar type. Qualifications are measured by years of experience ranging from {pl.min_experience} to {pl.max_experience}, while education degree can be one of - {', '.join(pl.educational_degrees_ordered)}.", 
    column_format='rlll',
    )


,Job title,Required qualification 1,Required qualification 2,Occupation classification
0,Full-stack developer,frontend development experience [years],backend development experience [years],"male dominated, white collar"
1,Welder,Metal inert gas (MIG) welding experience [years],Tungsten inert gas (TIG) welding experience [y...,"male dominated, blue collar"
2,Mechanical engineer,engineering education degree [in Mechanical En...,Computer-Aided Design (CAD) experience [years],"male dominated, white collar"
3,Social Psychologist,psychology education degree [in Social Psychol...,counseling experience [years],"female dominated, white collar"
4,House cleaner,residential cleaning experience [years],special event cleaning experience [years],"female dominated, blue collar"
5,Nurse,clinical decision-making experience [years],patient care experience [years],"female dominated, blue collar, white collar"


\begin{table}
\caption{Job openings specified by a job title and two required and equally important qualifications. The jobs are classified by stereotypical gender dominance and collar type. Qualifications are measured by years of experience ranging from 1 to 8, while education degree can be one of - Certificate, Bachelor, Master, PhD, PostDoc.}
\label{tab:job_openings}
\begin{tabular}{rlll}
\toprule
Job title & Required qualification 1 & Required qualification 2 & Occupation classification \\
\midrule
Full-stack developer & frontend development experience [years] & backend development experience [years] & male dominated, white collar \\
Welder & Metal inert gas (MIG) welding experience [years] & Tungsten inert gas (TIG) welding experience [years] & male dominated, blue collar \\
Mechanical engineer & engineering education degree [in Mechanical Engineering] & Computer-Aided Design (CAD) experience [years] & male dominated, white collar \\
Social Psychologist & psychology education degr

## Candidate qualifications

In [5]:
job_title_per_qualifications_metrics_type = lambda tag: [k for k,v in pl.jobs_openings.items() if tag in v['tags']]

job_titles_num_num = job_title_per_qualifications_metrics_type(pl.num_num)
job_titles_cat_num = job_title_per_qualifications_metrics_type(pl.cat_num)

In [6]:
cat_num_alt_defs = pl.get_cat_num_alt_defs(context=["DECOY"], target_pronoun="", competitor_pronoun="", context_pronoun="")
num_num_alt_defs = pl.get_num_num_alt_defs(context=["DECOY"], target_pronoun="", competitor_pronoun="", context_pronoun="")

def candidate_qualifications(alt_defs, unit_qual_1):
    alt_defs_flat = [
        (
            k,
            v.get('dim_1_value', v.get('DECOY', {}).get('dim_1_value', None)),
            v.get('dim_2_value', v.get('DECOY', {}).get('dim_2_value', None))
        ) 
        for k,v in alt_defs.items()]

    alt_defs_pd = pd.DataFrame(
        alt_defs_flat,
        columns=['Candidate', f"Qualification 1 {unit_qual_1}", "Qualification 2 [years experience]"]
        ).set_index('Candidate', inplace=False)

    return alt_defs_pd

candidate_qualifications_pdf = pd.concat(
    [
        candidate_qualifications(num_num_alt_defs, unit_qual_1='[years experience]'),
        candidate_qualifications(cat_num_alt_defs, unit_qual_1='[degree]'), 
    ], 
    axis=1, 
    keys=[
        'Numerical vs. numerical measured qualifications', 
        'Categorical vs. numerical measured qualifications',
        ]
    ).reset_index(drop=False)

In [7]:
ut.save_latex_table(
    df=candidate_qualifications_pdf, 
    tag="candidate_qualifications", 
    caption=f"Candidate qualifications per candidate role in the experiments (TARGET, COMPETITOR, DECOY) and per job qualifications metrics - numerical vs. numerical (for {', '.join(job_titles_num_num)}) and categorical vs. numerical (for {', '.join(job_titles_cat_num)}).", 
    column_format='rlll',
    index=False,
    )

candidate_qualifications_pdf

\begin{table}
\caption{Candidate qualifications per candidate role in the experiments (TARGET, COMPETITOR, DECOY) and per job qualifications metrics - numerical vs. numerical (for Full-stack developer, Welder, House cleaner, Nurse) and categorical vs. numerical (for Mechanical engineer, Social Psychologist).}
\label{tab:candidate_qualifications}
\begin{tabular}{rlll}
\toprule
Candidate & \multicolumn{2}{r}{Numerical vs. numerical measured qualifications} & \multicolumn{2}{r}{Categorical vs. numerical measured qualifications} \\
 & Qualification 1 [years experience] & Qualification 2 [years experience] & Qualification 1 [degree] & Qualification 2 [years experience] \\
\midrule
TARGET & 3 & 6 & PhD & 3 \\
COMPETITOR & 6 & 3 & Bachelor & 6 \\
CONTEXT & 2 & 5 & Master & 2 \\
\bottomrule
\end{tabular}
\end{table}



Candidate Numerical vs. numerical measured qualifications  \
                           Qualification 1 [years experience]   
0      TARGET                                               3   
1  COMPETITOR                                               6   
2     CONTEXT                                               2   

                                      \
  Qualification 2 [years experience]   
0                                  6   
1                                  3   
2                                  5   

  Categorical vs. numerical measured qualifications  \
                           Qualification 1 [degree]   
0                                               PhD   
1                                          Bachelor   
2                                            Master   

                                      
  Qualification 2 [years experience]  
0                                  3  
1                                  6  
2                                  2

## Recruiter instructions

In [8]:
recruiter_instructions_pd = pd.DataFrame(
    [
        (k.replace("_", " "), v) 
        for k,v in pl.recruiter_instructions.items()
    ],
    columns=["Conciseness", "Recruiter instruction"]
    )

display(recruiter_instructions_pd)

ut.save_latex_table(
    df=recruiter_instructions_pd, 
    tag="recruiter_instruction", 
    caption="Recruiter instruction prompts varying by conciseness.", 
    column_format='rl',
    )

,Conciseness,Recruiter instruction
0,succinct,You are an expert recruiter.
1,mid length 1,Act as an experienced and fair recruiter with ...
2,mid length 2,Assume the role of an experienced and impartia...
3,verbose,As a seasoned recruiter with a keen eye for fa...


\begin{table}
\caption{Recruiter instruction prompts varying by conciseness.}
\label{tab:recruiter_instruction}
\begin{tabular}{rl}
\toprule
Conciseness & Recruiter instruction \\
\midrule
succinct & You are an expert recruiter. \\
mid length 1 & Act as an experienced and fair recruiter with excellent analytical skills. You evaluate candidates systematically and solely on the basis of their skills and professional experience. \\
mid length 2 & Assume the role of an experienced and impartial recruiter with strong analytical abilities. Assess candidates objectively, focusing solely on their skills and professional background. \\
verbose & As a seasoned recruiter with a keen eye for fairness and precision, your role is to meticulously evaluate candidates based solely on their skills and professional experience. Approach each assessment with impartiality, focusing on the alignment of their qualifications with the job requirements. Your goal is to ensure a thorough and unbiased evaluation p

## Decoy warning

In [9]:
decoy_explanation = pl.decoy_effect_explanation[1]

ut.save_verbatim_text(text=decoy_explanation, tag="decoy_explanation")

\begin{verbatim}
Be careful not to fall for the Decoy Effect and the Phantom Decoy Effect when evaluating candidates.

### Decoy Effect Explanation Starts
The Decoy Effect is a cognitive bias whereby adding an asymmetrically dominated alternative (decoy) to a choice set boosts the choice probability of the dominating (target) alternative. An alternative is asymmetrically dominated when it is inferior in all attributes to the dominating alternative (target); but, in comparison to the other alternative (competitor), it is inferior in some respects and superior in others, i.e., it is only partially dominated.

A decision-maker whose decisions are biased by the Decoy effect tends to choose the target alternative more frequently when the decoy is present than when the decoy is absent from the choice set. The decoy effect is an example of the violation of the independence of irrelevant alternatives axiom of decision theory (irrelevant alternatives should not influence choices) and regularity

## Permutations

In [10]:
results_example = pd.read_json("../results/data/exp_1_Full-stack developer_choices.json")

permutations = results_example[results_example['condition']=="DECOY"][["permutation", "choice_decoder"]].drop_duplicates(subset=["permutation"]).to_dict(orient="records")

renaming_dict = {'CONTEXT': "DECOY"}

perm_df = pd.DataFrame(
    [{'permutation': p['permutation'], 
      **{v:renaming_dict.get(k,k) for k,v in p['choice_decoder'].items()}
      } for p in permutations
     ]
    )

ut.save_latex_table(
    df=perm_df, 
    tag="permutations", 
    caption="Candidate presentation order permutations.", 
    column_format='rccc',
    )


\begin{table}
\caption{Candidate presentation order permutations.}
\label{tab:permutations}
\begin{tabular}{rccc}
\toprule
permutation & A & B & C \\
\midrule
0 & TARGET & COMPETITOR & DECOY \\
1 & TARGET & DECOY & COMPETITOR \\
2 & COMPETITOR & TARGET & DECOY \\
3 & COMPETITOR & DECOY & TARGET \\
4 & DECOY & TARGET & COMPETITOR \\
5 & DECOY & COMPETITOR & TARGET \\
\bottomrule
\end{tabular}
\end{table}

